# Extracting Dates from Text Data

**Goal: Extract relevant infromation from messy medical data using regex.**

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

**List of some variants might be encountered in this dataset:**
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

After extracting the dates, we need to sort them in ascending chronological order according to the following **rules/assumptions:**
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Lookout for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, we find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

Output date format: **yyyy-mm-dd**

**Example Output:**

                raw_text                  date
    0    3/25/93 Total time of visit    1993-03-25
    1    6/18/85 Primary Care Doctor    1985-06-18
    2    plans to move as of 7/8/71     1971-07-08

In [18]:
import re

import numpy as np
import pandas as pd

In [19]:
# Read data
filepath = './data/process/dates.txt'
doc = []

with open(filepath) as file:
    for line in file:
        doc.append(line.strip())

df = pd.Series(doc)
df.name = 'raw_text'
df.head(10)

0           03/25/93 Total time of visit (in minutes):
1                         6/18/85 Primary Care Doctor:
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                  7 on 9/27/75 Audit C Score Current:
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                      .Per 7/06/79 Movement D/O note:
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                           3/7/86 SOS-10 Total Score:
9               (4/10/71)Score-1Audit C Score Current:
Name: raw_text, dtype: object

In [23]:
def date_sorter():
    mod_df = df
    
    # Regular expressions to extract various date formats
    r_month_text = r'(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*'    
    r_year = r'(?:[12][90]\d{2}|\d{2})'
    r_day = r'[0-3]?[0-9]'
    regexes = [
        r'({}{}{}{})'.format(r'(?:[1][012]|[1-9])[-/\s]', r_day, '[-/\s]', r_year), # mm-dd-yyyy
        r'({}{}{}{}{})'.format(r_month_text, r'[-\s.]?\s?', r_day, '[-,]\s?', r_year), # month-dd-yyyy
        r'({}{}{}{})'.format(r'\d{2}\s', r_month_text, r'[,.]?\s', r_year), # dd-month-yyyy
        r'({}{}{})'.format(r_month_text, r'\s\d{2}(?=\w{2})?,?\s', r_year), # month-dd[st|nd...]-yyyy
        r'({}{}{})'.format(r_month_text, r',?\s?', r_year), # month-yyyy
        r'({}{})'.format(r'(?<=[-\s])?[01]?\d[-/,]\s?', r_year), # mm-yyyy
        r'([12][90]\d{2})'
    ]

    # Loop over the list of regexes to extract all dates 
    extracted_series_list = []
    for i, regex in enumerate(regexes):
        extracted = mod_df.str.extract(regex, flags=re.IGNORECASE).dropna()
        ext_index = extracted.dropna().index
        mod_df = mod_df.drop(ext_index)
        extracted_series_list.append(extracted)
    
    # Join all extracted dataframes and replace month name by first three characters
    final_df = (
        pd.concat(extracted_series_list)
        .loc[:, 0]
        .str.replace('[a-z]+', lambda x: x.group(0)[:2])
    )
    
    # Fix remaining anomalies manually
    final_df[[461, 465]] = final_df.loc[[461, 465]].str.replace('\d{2},\s', '')
    
    # Convert all extracted dates to datetime
    final_df = pd.to_datetime(final_df)
    
    return final_df

In [25]:
date_sorter().head()

0   1993-03-25
1   1985-06-18
2   1971-07-08
3   1975-09-27
4   1996-02-06
Name: 0, dtype: datetime64[ns]